In [37]:
#import all library
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm

import sklearn

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler,PowerTransformer,Normalizer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
from sklearn.linear_model import Lasso, LassoCV
from sklearn.linear_model import ElasticNet, ElasticNetCV
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV

pd.set_option('display.max_columns', 300)
pd.set_option('display.max_rows', 300)

In [38]:
#Read CSV file
housing = pd.read_csv('../Data/regression_train.csv')
housing = housing[housing['resale_price'] < 1000000] 
housing.head()                 

,id,resale_price,mid_storey,floor_area_sqft,hdb_age,max_floor_lvl,commercial,market_hawker,multistorey_carpark,precinct_pavilion,total_dwelling_units,mall_within_500m,mall_within_1km,mall_within_2km,hawker_within_500m,hawker_within_1km,hawker_within_2km,hawker_food_stalls,hawker_market_stalls,bus_interchange,mrt_interchange,bus_stop_nearest_distance,pri_sch_nearest_distance,vacancy,pri_sch_affiliation,sec_sch_nearest_dist,affiliation,train,indicator_to_drop,mature,top_10_pri_sch,cutoff_point_rank,percent_rental_flat,percent_4room_and_above_flat,distance_for_hawker_and_mrt,distance_for_mall_and_mrt,flat_type_2 ROOM,flat_type_3 ROOM,flat_type_4 ROOM,flat_type_5 ROOM,flat_type_EXECUTIVE,flat_type_MULTI-GENERATION,flat_model_Adjoined flat,flat_model_Apartment,flat_model_DBSS,flat_model_Improved,flat_model_Improved-Maisonette,flat_model_Maisonette,flat_model_Model A,flat_model_Model A-Maisonette,flat_model_Model A2,flat_model_Multi Generation,flat_model_New Generation,flat_model_Premium Apartment,flat_model_Premium Apartment Loft,flat_model_Premium Maisonette,flat_model_Simplified,flat_model_Standard,flat_model_Terrace,flat_model_Type S1,flat_model_Type S2,mrt_name_Aljunied,mrt_name_Ang Mo Kio,mrt_name_Bartley,mrt_name_Beauty World,mrt_name_Bedok,mrt_name_Bedok North,mrt_name_Bedok Reservoir,mrt_name_Bencoolen,mrt_name_Bendemeer,mrt_name_Bishan,mrt_name_Boon Keng,mrt_name_Boon Lay,mrt_name_Botanic Gardens,mrt_name_Braddell,mrt_name_Bras Basah,mrt_name_Buangkok,mrt_name_Bugis,mrt_name_Bukit Batok,mrt_name_Bukit Gombak,mrt_name_Bukit Panjang,mrt_name_Buona Vista,mrt_name_Caldecott,mrt_name_Canberra,mrt_name_Cashew,mrt_name_Changi Airport,mrt_name_Chinatown,mrt_name_Chinese Garden,mrt_name_Choa Chu Kang,mrt_name_Clarke Quay,mrt_name_Clementi,mrt_name_Commonwealth,mrt_name_Dakota,mrt_name_Dover,mrt_name_Eunos,mrt_name_Farrer Park,mrt_name_Farrer Road,mrt_name_Geylang Bahru,mrt_name_HarbourFront,mrt_name_Hillview,mrt_name_Holland Village,mrt_name_Hougang,mrt_name_Jalan Besar,mrt_name_Jurong East,mrt_name_Kaki Bukit,mrt_name_Kallang,mrt_name_Kembangan,mrt_name_Khatib,mrt_name_Kovan,mrt_name_Labrador Park,mrt_name_Lakeside,mrt_name_Lavender,mrt_name_Little India,mrt_name_Lorong Chuan,mrt_name_MacPherson,mrt_name_Marsiling,mrt_name_Marymount,mrt_name_Mattar,mrt_name_Mountbatten,mrt_name_Nicoll Highway,mrt_name_Novena,mrt_name_Outram Park,mrt_name_Pasir Ris,mrt_name_Paya Lebar,mrt_name_Pioneer,mrt_name_Potong Pasir,mrt_name_Punggol,mrt_name_Queenstown,mrt_name_Redhill,mrt_name_Rochor,mrt_name_Sembawang,mrt_name_Sengkang,mrt_name_Serangoon,mrt_name_Simei,mrt_name_Tai Seng,mrt_name_Tampines,mrt_name_Tampines East,mrt_name_Tampines West,mrt_name_Tan Kah Kee,mrt_name_Tanah Merah,mrt_name_Tanjong Pagar,mrt_name_Telok Blangah,mrt_name_Tiong Bahru,mrt_name_Toa Payoh,mrt_name_Ubi,mrt_name_Upper Changi,mrt_name_Woodlands,mrt_name_Woodlands North,mrt_name_Woodlands South,mrt_name_Woodleigh,mrt_name_Yew Tee,mrt_name_Yio Chu Kang,mrt_name_Yishun,mrt_name_one-north,planning_area_Bedok,planning_area_Bishan,planning_area_Bukit Batok,planning_area_Bukit Merah,planning_area_Bukit Panjang,planning_area_Bukit Timah,planning_area_Changi,planning_area_Choa Chu Kang,planning_area_Clementi,planning_area_Downtown Core,planning_area_Geylang,planning_area_Hougang,planning_area_Jurong East,planning_area_Jurong West,planning_area_Kallang,planning_area_Marine Parade,planning_area_Novena,planning_area_Outram,planning_area_Pasir Ris,planning_area_Punggol,planning_area_Queenstown,planning_area_Rochor,planning_area_Sembawang,planning_area_Sengkang,planning_area_Serangoon,planning_area_Tampines,planning_area_Tanglin,planning_area_Toa Payoh,planning_area_Western Water Catchment,planning_area_Woodlands,planning_area_Yishun,tranc_yearmonth_2012-04,tranc_yearmonth_2012-05,tranc_yearmonth_2012-06,tranc_yearmonth_2012-07,tranc_yearmonth_2012-08,tranc_yearmonth_2012-09,tranc_yearmonth_2012-10,tranc_yearmonth_2012-11,tranc_yearmonth_2012-12,tranc_yearmonth_2013-01,tranc_yearmonth_2

In [3]:
X=housing.drop(columns=['resale_price','indicator_to_drop','train','vacancy','id'],inplace=False)
y=housing['resale_price']

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    np.log(y),
                                                    test_size = 0.2,
                                                    random_state = 42)

In [5]:
sklearn.metrics.SCORERS.keys()

dict_keys(['explained_variance', 'r2', 'max_error', 'matthews_corrcoef', 'neg_median_absolute_error', 'neg_mean_absolute_error', 'neg_mean_absolute_percentage_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_root_mean_squared_error', 'neg_mean_poisson_deviance', 'neg_mean_gamma_deviance', 'accuracy', 'top_k_accuracy', 'roc_auc', 'roc_auc_ovr', 'roc_auc_ovo', 'roc_auc_ovr_weighted', 'roc_auc_ovo_weighted', 'balanced_accuracy', 'average_precision', 'neg_log_loss', 'neg_brier_score', 'adjusted_rand_score', 'rand_score', 'homogeneity_score', 'completeness_score', 'v_measure_score', 'mutual_info_score', 'adjusted_mutual_info_score', 'normalized_mutual_info_score', 'fowlkes_mallows_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_s

In [6]:
# Set up a list of ridge alphas to check.
# np.logspace generates 100 values equally between 0 and 5,
# then converts them to alphas between 10^0 and 10^5.
r_alphas = np.logspace(0, 10, 1)

# Cross-validate over our list of ridge alphas.
ridge_cv = RidgeCV(alphas=r_alphas, scoring='max_error', cv=3)

# Fit model using best ridge alpha!
ridge_cv = ridge_cv.fit(X_train, y_train)

C:\Users\Jia Hui\Anaconda3\envs\dsi-sg\lib\site-packages\sklearn\linear_model\_ridge.py:212: LinAlgWarning: Ill-conditioned matrix (rcond=3.60965e-18): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
C:\Users\Jia Hui\Anaconda3\envs\dsi-sg\lib\site-packages\sklearn\linear_model\_ridge.py:212: LinAlgWarning: Ill-conditioned matrix (rcond=3.60861e-18): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
C:\Users\Jia Hui\Anaconda3\envs\dsi-sg\lib\site-packages\sklearn\linear_model\_ridge.py:212: LinAlgWarning: Ill-conditioned matrix (rcond=3.64501e-18): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
C:\Users\Jia Hui\Anaconda3\envs\dsi-sg\lib\site-packages\sklearn\linear_model\_ridge.py:212: LinAlgWarning: Ill-conditioned matrix (rcond=2.43466e-18): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


In [7]:
# Here is the optimal value of alpha
ridge_cv.alpha_

1.0

In [8]:
# Instantiate a StandardScaler + ridge pipeline.
pipe_r = Pipeline([
    ('ss', StandardScaler()),
    ('r', Ridge(alpha=1))
])

In [9]:
def regression_metrices(pipe):
    
    pipe.fit(X_train,y_train)
    
    print('r-score on train dataset:',pipe.score(X_train,y_train))
    print('r-score on test dataset:', pipe.score(X_test,y_test))
    
    y_pred = pipe.predict(X_test)
    
    # Evaluation metrics for the actual model
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred, squared = False)

    print("MAE:", mae)
    print("MSE:", mse)
    print("RMSE:", rmse)

In [10]:
regression_metrices(pipe_r)

r-score on train dataset: 0.9174875809155394
r-score on test dataset: 0.917537037642445
MAE: 0.0668827899948052
MSE: 0.007467044987609039
RMSE: 0.08641206505812159


### Lasso 

In [11]:
# Set up a list of ridge alphas to check.
# np.logspace generates 100 values equally between 0 and 5,
# then converts them to alphas between 10^0 and 10^5.
l_alphas = np.logspace(-3, 0, 10)

# Cross-validate over our list of ridge alphas.
lasso_cv = LassoCV(alphas=l_alphas, cv=5, max_iter=10000)

# Fit model using best ridge alpha!
lasso_cv = lasso_cv.fit(X_train, y_train)

In [12]:
# Here is the optimal value of alpha
lasso_cv.alpha_

0.001

In [13]:
# Instantiate a StandardScaler + ridge pipeline.
pipe_l = Pipeline([
    ('l', Lasso(alpha=0.21544346900318823))
])

In [14]:
regression_metrices(pipe_l)

r-score on train dataset: 0.7013078849615386
r-score on test dataset: 0.7038159391500796
MAE: 0.12991984470024445
MSE: 0.0268195519994737
RMSE: 0.1637667609726519


### ElasticNet

In [15]:
# Instantiate a StandardScaler + ridge pipeline.
pipe_e = Pipeline([
    ('e', ElasticNet(alpha=0.5))
])

In [16]:
regression_metrices(pipe_e)

r-score on train dataset: 0.6804573415513352
r-score on test dataset: 0.6830318530772872
MAE: 0.1343720189466368
MSE: 0.028701556978375127
RMSE: 0.16941533867503003


Ridge has the smallest RMSE and higest R^2

### OLS

In [17]:
X_train = sm.add_constant(X_train)
ols = sm.OLS(y_train,X_train)
result = ols.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           resale_price   R-squared:                       0.917
Model:                            OLS   Adj. R-squared:                  0.917
Method:                 Least Squares   F-statistic:                     4807.
Date:                Mon, 11 Mar 2024   Prob (F-statistic):               0.00
Time:                        21:00:54   Log-Likelihood:             1.2378e+05
No. Observations:              119596   AIC:                        -2.470e+05
Df Residuals:                  119319   BIC:                        -2.443e+05
Df Model:                         276                                         
Covariance Type:            nonrobust                                         
=========================================================================================================
                                            coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------
const                                    11.2370      0.041    273.865      0.000      11.157      11.317
mid_storey                                0.0069   5.51e-05    124.597      0.000       0.007       0.007
floor_area_sqft                           0.0006   4.55e-06    137.116      0.000       0.001       0.001
hdb_age                                  -0.0087   4.54e-05   -190.858      0.000      -0.009      -0.009
max_floor_lvl                             0.0042    8.2e-05     51.483      0.000       0.004       0.004
commercial                               -0.0036      0.001     -5.262      0.000      -0.005      -0.002
market_hawker                             0.0209      0.025      0.834      0.404      -0.028       0.070
multistorey_carpark                      -0.1129      0.015     -7.472      0.000      -0.142      -0.083
precinct_pavilion                         0.0750      0.009      8.732      0.000       0.058       0.092
total_dwelling_units                     -0.0002   6.43e-06    -34.930      0.000      -0.000      -0.000
mall_within_500m                          0.0103      0.000     21.614      0.000       0.009       0.011
mall_within_1km                          -0.0014      0.000     -4.008      0.000      -0.002      -0.001
mall_within_2km                           0.0024      0.000     14.765      0.000       0.002       0.003
hawker_within_500m                       -0.0028      0.001     -4.743      0.000      -0.004      -0.002
hawker_within_1km                         0.0017      0.000      4.035      0.000       0.001       0.002
hawker_within_2km                         0.0018      0.000      5.200      0.000       0.001       0.002
hawker_food_stalls                       -0.0003   2.02e-05    -14.928      0.000      -0.000      -0.000
hawker_market_stalls                      0.0003    7.5e-06     42.202      0.000       0.000       0.000
bus_interchange                           0.1095      0.004     28.453      0.000       0.102       0.117
mrt_interchange                           0.0870      0.003     32.476      0.000       0.082       0.092
bus_stop_nearest_distance             -5.466e-06   4.76e-06     -1.149      0.251   -1.48e-05    3.86e-06
pri_sch_nearest_distance              -2.784e-05   1.44e-06    -19.365      0.000   -3.07e-05    -2.5e-05
pri_sch_affiliation                       0.0064      0.001      5.592      0.000       0.004       0.009
sec_sch_nearest_dist                   1.897e-05   1.15e-06     16.534      0.000    1.67e-05    2.12e-05
affiliation                               0.0082      0.002      3.859      0.000       0.004       0.012
mature                                    1.0201      0.009    113.604      0.000       1.002       1.038
top_10_pr

In [18]:
d = {}
for i in X.columns.tolist():
    d[f'{i}'] = result.pvalues[i]

df_pvalue= pd.DataFrame(d.items(), columns=['Var_name', 'p-Value']).sort_values(by = 'p-Value').reset_index(drop=True)

In [19]:
df_pvalue_more_than_zero_point_five= df_pvalue[df_pvalue['p-Value']>0.5]

In [20]:
df_pvalue_more_than_zero_point_five

,Var_name,p-Value
281,flat_model_Maisonette,0.515908
282,flat_model_Apartment,0.632444
283,mrt_name_Eunos,0.764238
284,mrt_name_Chinese Garden,0.779746
285,mrt_name_Little India,0.783141
286,cutoff_point_rank,0.789255
287,mrt_name_Bukit Gombak,0.839348
288,mrt_name_Serangoon,0.961991


In [21]:
cols_to_drop = list(df_pvalue_more_than_zero_point_five['Var_name'].unique())

In [22]:
cols_to_drop

['flat_model_Maisonette',
 'flat_model_Apartment',
 'mrt_name_Eunos',
 'mrt_name_Chinese Garden',
 'mrt_name_Little India',
 'cutoff_point_rank',
 'mrt_name_Bukit Gombak',
 'mrt_name_Serangoon']

In [23]:
## Not going to drop market_hawker as I believe it is an important factor that the model should consider when predicting

In [24]:
cols_to_drop_list = ['flat_model_Maisonette',
 'flat_model_Apartment',
 'mrt_name_Eunos',
 'mrt_name_Chinese Garden',
 'mrt_name_Little India',
 'cutoff_point_rank',
 'mrt_name_Bukit Gombak',
 'mrt_name_Serangoon',
'resale_price','indicator_to_drop','train','vacancy','id']

In [25]:
X=housing.drop(columns= cols_to_drop_list ,inplace=False)
y=housing['resale_price']

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    np.log(y),
                                                    test_size = 0.2,
                                                    random_state = 42)

In [27]:
# Instantiate a StandardScaler + ridge pipeline.
pipe_r = Pipeline([
    ('ss', StandardScaler()),
    ('r', Ridge(alpha=1))
])

In [28]:
regression_metrices(pipe_r)

r-score on train dataset: 0.917057879627739
r-score on test dataset: 0.9171767531115049
MAE: 0.06708039371361962
MSE: 0.007499668855633623
RMSE: 0.0866006284944493


In [29]:
## By drop P- values lesser than 0.05 greatly decrease the RMSE and improve the R^2 better

### Finding X variable with huge impact on resale price

In [30]:
ridge_coefs = pipe_r.named_steps['r'].coef_

In [31]:
ridge_df= pd.DataFrame()
ridge_df['x'] = X.columns
ridge_df['coef']= ridge_coefs

In [32]:
ridge_df = ridge_df.sort_values(by='coef',)

In [33]:
# Top 5 biggest negative impact on resale price 
ridge_df[:5]

,x,coef
2,hdb_age,-0.103862
119,mrt_name_Sengkang,-0.049761
170,planning_area_Woodlands,-0.035635
28,distance_for_hawker_and_mrt,-0.033374
114,mrt_name_Punggol,-0.029982


In [34]:
# Top 5 biggest postitive impact on resale price 
ridge_df.tail()

,x,coef
31,flat_type_3 ROOM,0.093276
34,flat_type_EXECUTIVE,0.097553
33,flat_type_5 ROOM,0.143733
32,flat_type_4 ROOM,0.146898
1,floor_area_sqft,0.165481
